In [51]:
from confluent_kafka import Consumer
import uuid
import os
from dotenv import load_dotenv
load_dotenv()
import json
import re

In [52]:
KAFKA_SERVER = os.getenv('KAFKA_SERVER')
KAFKA_USERNAME=os.getenv('KAFKA_USERNAME')
KAFKA_PASSWORD=os.getenv('KAFKA_PASSWORD')
KAFKA_TOPIC_NAME = os.getenv('KAFKA_TOPIC_NAME')

In [53]:
c = Consumer({
    'bootstrap.servers': KAFKA_SERVER,
    'group.id': f'deleton' +str(uuid.uuid1()),
    'security.protocol': 'SASL_SSL',
    'sasl.mechanisms': 'PLAIN',
    'sasl.username': KAFKA_USERNAME,
    'sasl.password': KAFKA_PASSWORD,
    'session.timeout.ms': 6000,
    'heartbeat.interval.ms': 1000,
    'fetch.wait.max.ms': 6000,
    'auto.offset.reset': 'latest',
    'enable.auto.commit': 'false',
    'max.poll.interval.ms': '86400000',
    'topic.metadata.refresh.interval.ms': "-1",
    "client.id": 'id-002-005',
})

c.subscribe([KAFKA_TOPIC_NAME])

In [42]:
i = 0
while i < 10:
    kafka_message = c.poll(0.5)

    if kafka_message is not None:

        kafka_message_value = kafka_message.value().decode('utf-8')

        kafka_message_json = json.loads(kafka_message_value)

        print(kafka_message_value)

    i += 1


{"log": "2022-10-04 10:32:39.344776 mendoza v9: [INFO]: Telemetry - hrt = 122; rpm = 28; power = 5.413223856\n"}
{"log": "2022-10-04 10:32:39.845460 mendoza v9: [INFO]: Ride - duration = 145.0; resistance = 30\n"}
{"log": "2022-10-04 10:32:40.346146 mendoza v9: [INFO]: Telemetry - hrt = 125; rpm = 31; power = 6.415485154\n"}
{"log": "2022-10-04 10:32:40.846831 mendoza v9: [INFO]: Ride - duration = 146.0; resistance = 30\n"}
{"log": "2022-10-04 10:32:41.347504 mendoza v9: [INFO]: Telemetry - hrt = 126; rpm = 38; power = 8.962429663\n"}
{"log": "2022-10-04 10:32:41.848176 mendoza v9: [INFO]: Ride - duration = 147.0; resistance = 30\n"}
{"log": "2022-10-04 10:32:42.348849 mendoza v9: [INFO]: Telemetry - hrt = 126; rpm = 37; power = 8.582257545\n"}


In [50]:
i = 0
while i < 10:
    kafka_message = c.poll(0.5)

    if kafka_message is not None:

        kafka_message_value = kafka_message.value().decode('utf-8')

        kafka_message_json = json.loads(kafka_message_value)

        print(kafka_message_json['log'])

    i += 1

2022-10-04 10:33:14.392128 mendoza v9: [INFO]: Telemetry - hrt = 134; rpm = 35; power = 7.837601654

2022-10-04 10:33:14.892809 mendoza v9: [INFO]: Ride - duration = 180.0; resistance = 30

2022-10-04 10:33:15.393146 mendoza v9: [INFO]: Telemetry - hrt = 132; rpm = 35; power = 7.837601654

2022-10-04 10:33:15.893828 mendoza v9: [INFO]: Ride - duration = 181.0; resistance = 30

2022-10-04 10:33:16.394471 mendoza v9: [INFO]: Telemetry - hrt = 132; rpm = 31; power = 6.415485154

2022-10-04 10:33:16.895167 mendoza v9: [INFO]: Ride - duration = 182.0; resistance = 30

2022-10-04 10:33:17.395958 mendoza v9: [INFO]: Telemetry - hrt = 133; rpm = 34; power = 7.473425372

2022-10-04 10:33:17.896632 mendoza v9: [INFO]: Ride - duration = 183.0; resistance = 30

2022-10-04 10:33:18.397269 mendoza v9: [INFO]: Telemetry - hrt = 132; rpm = 36; power = 8.207264682

2022-10-04 10:33:18.897937 mendoza v9: [INFO]: Ride - duration = 184.0; resistance = 30



%4|1664880491.935|SESSTMOUT|id-002-005#consumer-10| [thrd:main]: Consumer group session timed out (in join-state steady) after 717285 ms without a successful response from the group coordinator (broker 9, last error was Success): revoking assignment and rejoining group
%5|1664880492.044|REQTMOUT|id-002-005#consumer-10| [thrd:sasl_ssl://b13-pkc-l6wr6.europe-west2.gcp.confluent.cloud:9092/]: sasl_ssl://b13-pkc-l6wr6.europe-west2.gcp.confluent.cloud:9092/13: Timed out FetchRequest in flight (after 717396ms, timeout #0)
%4|1664880492.052|REQTMOUT|id-002-005#consumer-10| [thrd:sasl_ssl://b13-pkc-l6wr6.europe-west2.gcp.confluent.cloud:9092/]: sasl_ssl://b13-pkc-l6wr6.europe-west2.gcp.confluent.cloud:9092/13: Timed out 1 in-flight, 0 retry-queued, 0 out-queue, 0 partially-sent requests
%3|1664880492.069|FAIL|id-002-005#consumer-10| [thrd:sasl_ssl://b13-pkc-l6wr6.europe-west2.gcp.confluent.cloud:9092/]: sasl_ssl://b13-pkc-l6wr6.europe-west2.gcp.confluent.cloud:9092/13: 1 request(s) timed out: 

In [61]:
i = 0
current_data = {}

def update_duration_and_resistance(current_data: dict, duration_and_resistance: list):
## Will update current data given [duration, resistance]
    duration = duration_and_resistance[0]
    resistance = duration_and_resistance[1]

    current_data['duration'] = duration
    current_data['resistance'] = resistance

def update_heart_rpm_and_power(current_data, heart_rpm_and_power):
## Will update current data given [heart, rpm, power]
    heart_rate = heart_rpm_and_power[0]
    rpm = heart_rpm_and_power[1]
    power = heart_rpm_and_power[2]

    current_data['heart_rate'] = heart_rate
    current_data['rpm'] = rpm
    current_data['power'] = power

while i < 10:
    kafka_message = c.poll(0.5) #poll all messages that have occurred in last 0.5 seconds

    if kafka_message is not None: #ensure we have a message
        log = kafka_message.value().decode('utf-8')

        if 'INFO' in log: #only check for strings with INFO
            values = json.loads(log)
            new_log = values.get('log')

            if 'Ride' in new_log: #process strings with Ride info
                new_log = new_log.split(' mendoza v9: [INFO]: Ride - ')
                timestamp = new_log[0]
                current_data['timestamp'] = timestamp

                duration_and_resistance = re.findall(r'\d+.?\d+',new_log[1])
                update_duration_and_resistance(current_data, duration_and_resistance)

            
            if 'Telemetry' in new_log:
                new_log = new_log.split(' mendoza v9: [INFO]: Telemetry - ')
                timestamp = new_log[0]
                current_data['timestamp'] = timestamp

                heart_rpm_and_power = re.findall(r'\d+.?\d+',new_log[1])
                update_heart_rpm_and_power(current_data, heart_rpm_and_power)
        
        if 'SYSTEM' in log:
            values = json.loads(log)
            new_log = values.get('log')


    print(current_data)
    
    i +=1

{}
{}
{'timestamp': '2022-10-04 11:51:33.341183', 'duration': '236.0', 'resistance': '30'}
{'timestamp': '2022-10-04 11:51:33.841860', 'duration': '236.0', 'resistance': '30', 'heart_rate': '116', 'rpm': '39', 'power': '9.347636596000001'}
{'timestamp': '2022-10-04 11:51:34.342536', 'duration': '237.0', 'resistance': '30', 'heart_rate': '116', 'rpm': '39', 'power': '9.347636596000001'}
{'timestamp': '2022-10-04 11:51:34.342536', 'duration': '237.0', 'resistance': '30', 'heart_rate': '116', 'rpm': '39', 'power': '9.347636596000001'}
{'timestamp': '2022-10-04 11:51:34.843230', 'duration': '237.0', 'resistance': '30', 'heart_rate': '115', 'rpm': '38', 'power': '8.962429663'}
{'timestamp': '2022-10-04 11:51:35.343912', 'duration': '238.0', 'resistance': '30', 'heart_rate': '115', 'rpm': '38', 'power': '8.962429663'}
{'timestamp': '2022-10-04 11:51:35.343912', 'duration': '238.0', 'resistance': '30', 'heart_rate': '115', 'rpm': '38', 'power': '8.962429663'}
{'timestamp': '2022-10-04 11:51:3